In [1]:
import numpy as np
import os
import threading
import operator
import pickle
import re
import pandas as pd


start_time = 1190146243
end_time = 1192994591

def load_data(input_name):
    with open(input_name, 'rb') as f:
        data = pickle.load(f)
    return data
def dump_data(data, output_name):
    with open(output_name, 'wb') as f:
        pickle.dump(data, f)

In [3]:
from scipy.sparse import coo_matrix

In [4]:
link_matrix = coo_matrix((100000, 100000))

In [5]:
word_freq = load_data('data/dump/hot_words_100000.pkl')

In [36]:
from urllib.parse import unquote
from urllib.parse import quote

def unquote_word(word):
    word = " ".join(unquote(unquote(word)).split("_"))
    return word

num_to_word = list(set(list(map(lambda x: unquote_word(x),list(set(word_freq.index))))))
word_to_num = {word:i  for word,i in zip(num_to_word, range(len(num_to_word)))}

True

In [64]:
from ipywidgets import IntProgress


links_to = []
in_degree = {}
out_degree = {}
freq = {}

for word in num_to_word:
    in_degree[word] = 0
    out_degree[word] = 0

files = [name for name in os.listdir("data/dump") if 'metadata' in name]
p = IntProgress(max = 100000)

display(p)
for file in files:
    meta = load_data("data/dump/" + file)
    for word in meta:
        unquoted = unquote_word(word)
        freq[unquoted] = word_freq[word]
        record = meta[word]
        link = record['links']
        for to_word in link:
            if to_word in word_to_num:
                in_degree[to_word] += 1
                out_degree[unquoted] += 1
                links_to.append((unquoted, to_word))
                
        p.value += 1

dump_data((links_to, in_degree, out_degree), "data/dump/link_records_in_degree_out_degree.pkl")

IntProgress(value=0, max=100000)

In [65]:
meta_df = pd.concat([pd.Series(in_degree),pd.Series(out_degree), pd.Series(freq)], axis = 1).dropna(axis=0, how='any')
meta_df = meta_df.rename(columns={0: 'in_degree', 1:'out_degree', 2:'freq'})

In [66]:
meta_df

,in_degree,out_degree,freq
!!!,10,29,1180.0
"""Heroes""",0,1,1243.0
"""Manos"" The Hands of Fate",0,0,751.0
"""Superstar"" Billy Graham",1,0,852.0
"""The Spaghetti Incident?""",45,87,1522.0
"""Weird Al"" Yankovic",230,285,9176.0
"""Weird Al"" Yankovic (album)",25,111,809.0
"""Weird Al"" Yankovic in 3-D",27,108,789.0
$1,0,11,7791.0
"$10,000 unsecured visa or mastercards",0,0,980.0


In [67]:
#dump_data(meta_df, "data/dump/graph_info.pkl")

In [72]:
with open("data/dump/link_info.txt", 'w') as f:
    for record in links_to:
        f.write("{} {}\n".format(word_to_num[record[0]], word_to_num[record[1]]))
with open("data/dump/word_index.txt", 'w') as f:
    for word in word_to_num:
        f.write("{} {}\n".format(word, word_to_num[word]))

In [4]:
metadf = load_data("data/dump/merged_metadata.pkl")

In [5]:
metadf.head()

,category_num,frequency,image_num,language_num,link_num,update,vandalism,in_degree,out_degree,page_len
$1,4.0,7791.0,1.0,1.0,45.0,0.0,0.0,0.0,11.0,2476.0
"$10,000_unsecured_visa_or_mastercards",0.0,980.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'O_Sole_Mio,0.0,897.0,0.0,0.0,1.0,0.0,0.0,7.0,0.0,25.0
(Don't_Fear)_The_Reaper,14.0,970.0,9.0,9.0,443.0,17.0,0.0,40.0,170.0,26292.0
(I_Can't_Get_No)_Satisfaction,49.0,667.0,11.0,31.0,590.0,10.0,0.0,71.0,228.0,37360.0


In [19]:
from pydoc import help
from scipy.stats.stats import pearsonr
import numpy as np
pearsonr(np.log(metadf['in_degree'] + 1),np.log(metadf['out_degree'] +1))

(0.7475740313653426, 0.0)

'0.0000000000'